In [352]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.common.exceptions import NoSuchElementException,ElementNotInteractableException
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait


In [353]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'}

options=webdriver.EdgeOptions()
# Add the arguments for Edge
options.add_argument("-inprivate")  # For private browsing in Edge
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--disable-cache")
options.add_argument("--disable-cookies")

In [354]:
url="https://super.walmart.com.mx/"
driver=webdriver.Edge(options=options)
driver.get(url)
time.sleep(5)

In [355]:
try:
    elemento = driver.find_element(By.NAME, 'select-store-button')
    
except NoSuchElementException:
    elemento = driver.find_element(By.CSS_SELECTOR, 'img[src="https://i5.walmartimages.com/dfw/4ff9c6c9-14f0/k2-_8ce34043-6050-4f39-afb2-3e76a554f8a3.v1.png"]')

elemento.click()

In [356]:
#Código postal
time.sleep(4)
cp=driver.find_element(By.CSS_SELECTOR,'input[data-automation-id="store-zip-code"]')
cp.clear()
cp.send_keys('58090')
time.sleep(4)
radio_buttons = driver.find_element(By.NAME,'pickup-store')
radio_buttons.click()
#Elegir
elegir=driver.find_element(By.CSS_SELECTOR,'button[data-automation-id="save-label"]')
elegir.click()

In [357]:
time.sleep(5)
url_busqueda="https://super.walmart.com.mx/search?q=huevo+rojo"
driver.get(url_busqueda)
time.sleep(5)
html=driver.page_source


In [358]:
# #Ingresar búsqueda
# try:
#     wait = WebDriverWait(driver, 10)
#     busca = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'input[placeholder="Buscar en Walmart"]')))
#     driver.execute_script("arguments[0].value = '';", busca)
#     busca.send_keys(Keys.ENTER)
#     busca.send_keys('huevo')
#     busca.send_keys(Keys.ENTER)
# except NoSuchElementException:
#     busca = driver.find_element(By.CSS_SELECTOR, 'input[placeholder="Buscar en Walmart"]')
#     time.sleep(4)
#     busca.send_keys('huevo')
#     busca.send_keys(Keys.ENTER)


# try:
#     # Wait for the element to be clickable
#     wait = WebDriverWait(driver, 10)
#     busca = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'input[placeholder="Buscar en Walmart"]')))
#     driver.execute_script("arguments[0].value = '';", busca)
#     busca.send_keys(Keys.ENTER)
#     busca.send_keys('huevo')
#     busca.send_keys(Keys.ENTER)
        
# except ElementNotInteractableException:
#     time.sleep(4)
#     # busca = driver.find_element(By.CSS_SELECTOR, 'input[placeholder="Buscar en Walmart"]')
#     # busca.clear()
#     busca.send_keys('huevo')
#     busca.send_keys(Keys.ENTER)




# wait = WebDriverWait(driver, 10)
# busca = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'input[placeholder="Buscar en Walmart"]')))
# busca.send_keys('limon')
# busca.send_keys(Keys.ENTER)

In [359]:
soup = BeautifulSoup(html, 'html.parser')

In [360]:
results = soup.find_all('div', {'data-stack-index': '0'})

In [361]:
span_tags = []
product_titles = []

# Loop through each individual result and select the desired elements
for result in results:
    span_tags.extend(result.select('div[data-automation-id="product-price"] span'))
    product_titles.extend([title.text for title in result.select('span[data-automation-id="product-title"]')])

precios = []

for span in span_tags:
    text = span.text
    if 'precio actual' in text:
        precios.append(text.split("precio actual")[1].strip())

In [362]:
len(product_titles), len(precios)

(44, 44)

In [363]:
#Llevar product_title y precios a un dataframe
df=pd.DataFrame({'producto':product_titles,'precio':precios})
df

,producto,precio
0,Huevo rojo Bachoco fresco 12 pzas,$44.00
1,Huevo rojo Bachoco fresco 30 pzas,$95.50
2,Huevo rojo Bachoco fresco 18 pzas,$60.50
3,Huevo blanco San Juan 12 pzas,$33.50
4,Huevo blanco San Juan 30 pzas,$76.00
5,Huevo blanco San Juan 18 pzas,$49.00
6,Huevo blanco Bachoco fresco 12 pzas,$38.00
7,Clara de huevo San Juan líquida 500 g,$26.50
8,Huevo blanco Bachoco fresco 18 pzas,$52.00
9,Claras de huevo San Juan líquidas pasteurizada...,$46.50
